In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os, sys
%matplotlib inline

os.chdir('..\..\..\..')

from input_configuration import base_year

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
working_dir = r'outputs/agg/dash'
pd.options.display.float_format = '{:0,.2f}'.format
format_percent = "{:,.2%}".format

In [3]:
def load_data(fname):
    """ open and join model and survey data """

    df_model = pd.read_csv(os.path.join(working_dir,fname+'.csv'))
    df_model['source'] = 'model'
    df_survey = pd.read_csv(os.path.join(working_dir,'survey',fname+'.csv'))
    df_survey['source'] = 'survey'
    df = df_model.append(df_survey)

    return df

In [4]:
parcel_geog = pd.read_sql_table('parcel_'+base_year+'_geography', 'sqlite:///inputs/db/soundcast_inputs.db',
        columns=['CountyName','ParcelID'])

In [5]:
def dist_by_mode(purpose=None, county_filter='', neg_county_filter=False):
    df = load_data('tour_distance')
    if county_filter!='' :
        if neg_county_filter:
            df = df.loc[df.person_county!=county_filter]
        else:
            df = df.loc[df.person_county==county_filter]            
    # Trim any unreasonable survey distances off by taking max distance as max modeled bin value
    max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
    df = df.loc[df['tautodist_bin'] <= max_model_dist]
    df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
    df_dist = pd.DataFrame()
    for source in ['model','survey']:
        if purpose is not None:
            _df = df[(df['source'] == source) & (df['pdpurp'] == purpose)]
        else:
            _df = df[(df['source'] == source)]
        df1 = _df.groupby('tmodetp').sum()[['wt_autodist']].reset_index()
        df2 = _df.groupby('tmodetp').sum()[['toexpfac']].reset_index ()
        _df = df1.merge(df2, on='tmodetp')
        _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
        _df['source'] = source
        df_dist = df_dist.append(_df)
    df = df_dist.pivot_table(index='tmodetp', columns='source', aggfunc='first', values='commute_distance')
    df.rename(columns={'tmodetp': 'Tour Mode'}, inplace=True)
    for col in ['School Bus',' Park']:
        if (col in df.index) & (purpose != 'School'):
            df.drop(col, axis=0, inplace=True)
        if (col != 'School Bus') & (col in df.index):
            df.drop(col, axis=0, inplace=True)
    
    return df

### Total

In [6]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.67
survey,4.28


## Pierce County

In [7]:
df = load_data('tour_distance')
df = df.loc[df['person_county']=='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.62
survey,4.11


## Other Counties

In [8]:
df = load_data('tour_distance')
df = df.loc[df['person_county']!='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.69
survey,4.33


### By Purpose

In [9]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True)
df

source,model,survey
pdpurp,,
Escort,6.96,3.88
Meal,5.34,4.22
Personal Business,6.77,5.49
School,3.67,4.28
Shop,4.79,4.53
Social,4.60,4.89
Work,11.54,11.51


## Pierce County

In [10]:
df = load_data('tour_distance')
df = df.loc[df['person_county']=='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True, errors='ignore')
df

source,model,survey
pdpurp,,
Escort,6.77,4.71
Meal,5.78,5.53
Personal Business,6.88,6.91
School,3.62,4.11
Shop,5.19,4.45
Social,4.39,3.90
Work,13.76,12.67


## Other Counties

In [11]:
df = load_data('tour_distance')
df = df.loc[df['person_county']!='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True, errors='ignore')
df

source,model,survey
pdpurp,,
Escort,7.02,3.62
Meal,5.26,3.88
Personal Business,6.74,5.18
School,3.69,4.33
Shop,4.69,4.55
Social,4.65,5.10
Work,11.07,11.23


# Tour Distance
### All Tours

In [12]:
dist_by_mode()

source,model,survey
tmodetp,,
Bike,5.72,3.03
HOV2,7.24,6.70
HOV3+,6.88,6.22
Park,19.64,24.39
SOV,8.74,8.50
TNC,2.54,3.99
Transit,9.78,9.40
Walk,0.94,0.89


## Pierce County

In [13]:
dist_by_mode(county_filter='Pierce')

source,model,survey
tmodetp,,
Bike,5.90,4.78
HOV2,7.30,7.04
HOV3+,6.78,6.74
Park,22.95,33.47
SOV,9.28,8.72
TNC,2.71,10.34
Transit,20.18,15.27
Walk,0.99,0.92


## Other Counties

In [14]:
dist_by_mode(county_filter='Pierce', neg_county_filter=True)

source,model,survey
tmodetp,,
Bike,5.68,2.84
HOV2,7.23,6.61
HOV3+,6.91,6.09
Park,19.51,23.03
SOV,8.60,8.44
TNC,2.51,3.60
Transit,8.79,8.96
Walk,0.92,0.88


### Average Work Tour Distance By Mode

In [15]:
dist_by_mode('Work')

source,model,survey
tmodetp,,
Bike,6.05,4.31
HOV2,11.55,11.35
HOV3+,11.94,15.01
Park,19.64,23.26
SOV,12.13,11.78
TNC,6.19,4.86
Transit,13.06,10.77
Walk,0.76,1.29


## Pierce County

In [16]:
dist_by_mode('Work', 'Pierce')

source,model,survey
tmodetp,,
Bike,6.11,4.13
HOV2,12.86,12.28
HOV3+,13.20,13.14
Park,22.95,33.24
SOV,13.67,12.32
TNC,7.25,28.52
Transit,26.23,24.67
Walk,0.88,1.12


## Other Counties

In [17]:
dist_by_mode('Work', 'Pierce',True)

source,model,survey
tmodetp,,
Bike,6.04,4.33
HOV2,11.25,11.09
HOV3+,11.65,15.50
Park,19.51,21.62
SOV,11.78,11.62
TNC,6.04,4.21
Transit,11.41,9.78
Walk,0.75,1.32


### Average School Tour Distance by Mode

In [18]:
dist_by_mode('School')

source,model,survey
tmodetp,,
Bike,2.79,1.30
HOV2,3.54,5.36
HOV3+,3.65,3.62
Park,NaN,28.46
SOV,7.34,6.08
School Bus,3.20,3.46
TNC,2.35,2.77
Transit,5.71,7.66
Walk,1.11,0.95


## Pierce County

In [19]:
dist_by_mode('School', 'Pierce')

source,model,survey
tmodetp,,
Bike,3.11,NaN
HOV2,3.51,3.02
HOV3+,3.62,4.66
SOV,7.29,5.68
School Bus,3.16,2.94
TNC,2.34,NaN
Transit,4.83,16.75
Walk,1.15,1.32


## Other Counties

In [20]:
dist_by_mode('School', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,2.72,1.30
HOV2,3.54,5.73
HOV3+,3.66,3.34
Park,NaN,28.46
SOV,7.35,6.24
School Bus,3.22,3.66
TNC,2.35,2.77
Transit,5.82,7.50
Walk,1.11,0.91


### Average Personal Business Tour Distance by Mode

In [21]:
dist_by_mode('Personal Business')

source,model,survey
tmodetp,,
Bike,3.65,3.05
HOV2,7.80,6.48
HOV3+,7.81,6.01
Park,NaN,19.28
SOV,6.61,5.33
TNC,2.01,4.93
Transit,6.72,6.09
Walk,1.18,0.90


## Pierce County

In [22]:
dist_by_mode('Personal Business', 'Pierce')

source,model,survey
tmodetp,,
Bike,3.76,1.50
HOV2,7.86,9.82
HOV3+,7.79,10.78
Park,NaN,35.50
SOV,6.63,4.50
TNC,2.37,5.50
Transit,8.56,2.61
Walk,1.26,0.67


## Other Counties

In [23]:
dist_by_mode('Personal Business', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,3.62,3.16
HOV2,7.78,5.76
HOV3+,7.82,5.25
Park,NaN,12.63
SOV,6.60,5.55
TNC,1.93,4.86
Transit,6.65,6.78
Walk,1.16,0.94


### Average Shopping Tour Distance by Mode

In [24]:
dist_by_mode('Shop')

source,model,survey
tmodetp,,
Bike,2.64,0.56
HOV2,5.53,4.92
HOV3+,5.70,6.50
Park,NaN,0.50
SOV,4.76,3.76
TNC,1.45,2.64
Transit,4.39,2.26
Walk,1.07,0.63


## Pierce County

In [25]:
dist_by_mode('Shop', 'Pierce')

source,model,survey
tmodetp,,
Bike,3.08,0.50
HOV2,5.88,5.04
HOV3+,5.92,4.48
SOV,5.08,4.17
TNC,1.62,3.50
Transit,4.42,2.30
Walk,1.19,0.57


## Other Counties

In [26]:
dist_by_mode('Shop', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,2.53,0.56
HOV2,5.43,4.86
HOV3+,5.63,6.84
Park,NaN,0.50
SOV,4.68,3.62
TNC,1.42,0.89
Transit,4.39,2.25
Walk,1.04,0.63


### Average Meal Tour Distance by Mode

In [27]:
dist_by_mode('Meal')

source,model,survey
tmodetp,,
Bike,1.72,2.68
HOV2,8.68,4.59
HOV3+,9.23,6.29
Park,NaN,30.11
SOV,5.03,4.50
TNC,1.40,2.34
Transit,5.84,8.47
Walk,0.65,0.70


## Pierce County

In [28]:
dist_by_mode('Meal', 'Pierce')

source,model,survey
tmodetp,,
Bike,1.83,NaN
HOV2,8.60,4.83
HOV3+,9.01,10.03
Park,NaN,35.50
SOV,5.01,3.51
TNC,1.43,7.50
Transit,5.09,NaN
Walk,0.69,0.76


## Other Counties

In [29]:
dist_by_mode('Meal', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,1.70,2.68
HOV2,8.69,4.53
HOV3+,9.27,4.82
Park,NaN,14.85
SOV,5.04,4.88
TNC,1.40,2.04
Transit,5.88,8.47
Walk,0.64,0.70


### Average Social Tour Distance by Mode

In [30]:
dist_by_mode('Social')

source,model,survey
tmodetp,,
Bike,2.81,2.33
HOV2,6.13,7.38
HOV3+,6.13,4.98
Park,NaN,54.15
SOV,5.48,5.65
TNC,1.47,3.02
Transit,5.21,11.23
Walk,0.96,0.88


## Pierce County

In [31]:
dist_by_mode('Social', 'Pierce')

source,model,survey
tmodetp,,
Bike,2.72,6.87
HOV2,5.89,7.90
HOV3+,5.82,3.63
SOV,5.24,6.29
TNC,1.54,NaN
Transit,5.11,4.77
Walk,0.96,0.93


## Other Counties

In [32]:
dist_by_mode('Social', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,2.83,1.71
HOV2,6.19,7.32
HOV3+,6.21,5.32
Park,NaN,54.15
SOV,5.54,5.52
TNC,1.46,3.02
Transit,5.21,11.44
Walk,0.96,0.87


### Average Escort Tour Distance by Mode

In [33]:
dist_by_mode('Escort')

source,model,survey
tmodetp,,
Bike,13.41,0.65
HOV2,6.41,4.36
HOV3+,7.00,3.75
SOV,21.01,3.97
TNC,1.66,0.50
Transit,2.74,2.26
Walk,0.98,0.55


## Pierce County

In [34]:
dist_by_mode('Escort', 'Pierce')

source,model,survey
tmodetp,,
Bike,13.11,NaN
HOV2,6.22,3.68
HOV3+,6.66,5.71
SOV,22.20,5.35
TNC,1.70,NaN
Transit,2.33,0.50
Walk,1.02,1.00


## Other Counties

In [35]:
dist_by_mode('Escort', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,13.48,0.65
HOV2,6.47,4.60
HOV3+,7.09,3.10
SOV,20.65,3.41
TNC,1.65,0.50
Transit,2.78,2.35
Walk,0.97,0.55


Survey Sample Size Reference by Mode and Purpose

In [36]:
# Survey Sample size
df_tour_survey = pd.read_csv(r'inputs/base_year/survey/_tour.tsv', delim_whitespace=True)

In [37]:
df_tour_survey.pivot_table(index='tmodetp',columns='pdpurp', values='toexpfac', aggfunc='count')

pdpurp,1,2,3,4,5,6,7,10
tmodetp,,,,,,,,
1,889.00,189.00,158.00,375.00,631.00,811.00,"1,613.00",NaN
2,234.00,38.00,8.00,19.00,19.00,19.00,81.00,NaN
3,"3,215.00",114.00,43.00,890.00,820.00,352.00,937.00,1.00
4,945.00,284.00,637.00,560.00,587.00,373.00,743.00,NaN
5,455.00,442.00,477.00,336.00,287.00,246.00,659.00,1.00
6,"2,293.00",257.00,26.00,253.00,146.00,115.00,314.00,1.00
7,101.00,2.00,NaN,7.00,1.00,4.00,10.00,NaN
8,9.00,253.00,1.00,20.00,NaN,NaN,7.00,NaN
9,119.00,12.00,1.00,29.00,6.00,42.00,49.00,NaN


Survey Sample Size Reference by Mode and Purpose 
(for Pierce County)

In [38]:
df_household = pd.read_csv(r'inputs/base_year/survey/_household.tsv', delim_whitespace=True)
df_household = df_household[['hhno','hhtaz']]

pierce_tazs = [i for i in range(2669, 3466+1)]
king_tazs = [i for i in range(1, 2088+1)]
kitsap_tazs = [i for i in range(3467, 3700+1)]
snohomish_tazs = [i for i in range(2089, 2668+1)]
df_household.loc[df_household['hhtaz'].isin(pierce_tazs), 'county'] = 'Pierce'
df_household.loc[df_household['hhtaz'].isin(king_tazs), 'county'] = 'King'
df_household.loc[df_household['hhtaz'].isin(kitsap_tazs), 'county'] = 'Kitsap'
df_household.loc[df_household['hhtaz'].isin(snohomish_tazs), 'county'] = 'Snohomish'

In [39]:
df_tour_survey = pd.merge(df_tour_survey, df_household, on="hhno", how='left')
for county, df_county in df_tour_survey.groupby('county'):
    print(county + ' County\n')
    print(df_tour_survey.pivot_table(index=['tmodetp'],columns='pdpurp', values='toexpfac', aggfunc='count', fill_value=0))
    print('\n')

King County

pdpurp     1    2    3    4    5    6     7   10
tmodetp                                         
1         889  189  158  375  631  811  1613   0
2         234   38    8   19   19   19    81   0
3        3215  114   43  890  820  352   937   1
4         945  284  637  560  587  373   743   0
5         455  442  477  336  287  246   659   1
6        2293  257   26  253  146  115   314   1
7         101    2    0    7    1    4    10   0
8           9  253    1   20    0    0     7   0
9         119   12    1   29    6   42    49   0
10        128   27    8   32   17   11    47   0


Kitsap County

pdpurp     1    2    3    4    5    6     7   10
tmodetp                                         
1         889  189  158  375  631  811  1613   0
2         234   38    8   19   19   19    81   0
3        3215  114   43  890  820  352   937   1
4         945  284  637  560  587  373   743   0
5         455  442  477  336  287  246   659   1
6        2293  257   26  253  146  115 

# Work Tour Distance by Worker Type

In [40]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,11.98,12.15
1,Part-Time Worker,8.80,7.61
2,University Student,9.72,9.32


## Pierce County

In [41]:
df = load_data('tour_distance')
df = df.loc[df.person_county=='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,14.46,14.40
1,Part-Time Worker,9.39,5.25
2,University Student,11.06,15.19


## Other Counties

In [42]:
df = load_data('tour_distance')
df = df.loc[df.person_county!='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,11.46,11.66
1,Part-Time Worker,8.68,8.83
2,University Student,9.40,8.41
